In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import scipy.stats as stats

## inference for discrete data p. 54

1000 randomly selected adults responding to questions about how many dogs they own. Q: what's the 95% confidence interval for the average number of dogs in the population

In [2]:
df = pd.DataFrame(data={'n_dogs':[0,1,2,3,4], 'n_ppl':[600,300,50,30,20]})

In [3]:
df

,n_dogs,n_ppl
0,0,600
1,1,300
2,2,50
3,3,30
4,4,20


Use formula $\overline{y} = \frac{1}{n}\sum_{i=1}^{n}y_{i}$ where $n=1000$, $y_i = \textrm{n_dogs}\times\textrm{n_ppl}$  

In [4]:
sum(df['n_dogs']*df['n_ppl'])/1000

0.57

In [5]:
mean = 0.57

This is the average number of dogs per person, as agreed with R

Use formula $s_{y} = \sqrt{\frac{1}{n-1}\sum_{i=1}^{n}(y_{i}-\overline{y})^2}$

In [6]:
std = np.sqrt(sum(np.power((df['n_dogs']-0.57),2)*df['n_ppl'])/999)

In [7]:
std

0.8751376268141291

Also matches with the R output

Standard Error is $se = \frac{\sigma}{\sqrt{n}}$

In [8]:
se = std/np.sqrt(1000)

In [9]:
se

0.027674281668470926

95% confidence interval is based on a t-distribution with n-1 degrees of freedom (dof)

`stats.t.ppf` seems to be the equivalent of `qt` in R

In [10]:
stats.t.ppf?

Signature: stats.t.ppf(q, *args, **kwds)
Docstring:
Percent point function (inverse of `cdf`) at q of the given RV.

Parameters
----------
q : array_like
    lower tail probability
arg1, arg2, arg3,... : array_like
    The shape parameter(s) for the distribution (see docstring of the
    instance object for more information)
loc : array_like, optional
    location parameter (default=0)
scale : array_like, optional
    scale parameter (default=1)

Returns
-------
x : array_like
    quantile corresponding to the lower tail probability q.
File:      ~/miniconda3/envs/ros/lib/python3.8/site-packages/scipy/stats/_distn_infrastructure.py
Type:      method


In [11]:
int_95 = mean+stats.t.ppf([0.025, 0.975], 1000-1)*se

In [12]:
int_95

array([0.51569361, 0.62430639])

This looks correct

# Exercises

## 4.1 Comparison of proportions

In [13]:
#average treatment effect
estimate = 0.5-0.4
estimate

0.09999999999999998

In [14]:
#standard error of the treatment effect, see standard error for a comparison
se_ctrl = np.sqrt(0.4*0.6/500)
se_treat = np.sqrt(0.5*0.5/500)
se = np.sqrt(np.power(se_ctrl, 2)+np.power(se_treat,2))

se

0.03130495168499706

Answer: the estimated treatment effect is 0.1$\pm$0.03

## 4.2 Choosing sample size

Note that $se_{\textrm{tot}} = \sqrt{se_{1}^2+se_{2}^2}$, and $se = \sqrt{\frac{\hat{p}(1-\hat{p})}{n}}$, so $se_{\textrm{tot}} = \sqrt{\frac{\hat{p}_1\left(1-\hat{p}_1\right)}{N/2}+\frac{\hat{p}_2\left(1-\hat{p}_2\right)}{N/2}}$, if we assume that men and women are making up 0.5 of the total population

Since we are asking about supporting a candidate, we can assme 0.5 for $\hat{p}$ for both men and women as a mean since the response is either 0 or 1. Then we have

$se_{\textrm{tot}}^2 = \frac{4*0.5^2}{N} \leq 0.05^2 \implies \frac{4*0.5^2}{0.05^2} \leq N \implies 400 \leq N$

In [15]:
4*0.5*0.5/0.05/0.05

400.0

Or, you think, $se = \frac{\sigma}{\sqrt{N}}$, for a general population $\sigma = 1$, so $1/\sqrt{N} \leq 0.05 \implies N \geq 0.05^{-2} = 400$ 

In [16]:
np.power((1/0.05),2)

400.0

## 4.3 Comparison of Proportions

The question is asking for a p-value, with Null hypothesis that the better shooter makes more shots

average (mean) treatment effect is 0.1

standard error is

In [74]:
se = se_ctrl = np.sqrt(0.4*0.6/20)
se_treat = np.sqrt(0.3*0.7/20)
se = np.sqrt(np.power(se_ctrl, 2)+np.power(se_treat,2))

se

0.15

We know for a normal distribution $z=\frac{x-\mu}{\sigma/\sqrt{N}}$, we are trying to find the p-value. Here we know that if $x=0$, then the z-score would be for all the cdf of a Normal distribution up to 0, which means the cdf for this z-score gives the probability mass for us seeing $x\leq0$. So we need $1-\textrm{cdf}(z)$

In [76]:
z = (0-0.1)/(0.15)
z

-0.6666666666666667

In [77]:
1-stats.norm.cdf(z)

0.7475074624530771

So 75% chance of seeing a mean > 0 with normal

But the book talked about t tests. And our N is way too small to use a normal distribution, so what does the t look like?

In [78]:
1-stats.t.cdf(-0.1/0.15, df=40)

0.7455937742565715

for a t distribution it appears that we will see the effect 74% of the time, which looks very close

In [23]:
import pymc3 as pm

In [58]:
with pm.Model() as model:
    n1 = pm.Binomial('n1',p=0.4,n=20)
    n2 = pm.Binomial('n2',p=0.3,n=20)
    diff = pm.Deterministic('diff', n1-n2)
    
    trace = pm.sample(300000)

Multiprocess sampling (3 chains in 3 jobs)
CompoundStep
>Metropolis: [n2]
>Metropolis: [n1]


Sampling 3 chains for 1_000 tune and 300_000 draw iterations (3_000 + 900_000 draws total) took 88 seconds.
The number of effective samples is smaller than 25% for some parameters.


In [59]:
burned_trace = trace[10000:]

In [60]:
burned_trace['n1']

array([ 5,  5,  5, ..., 10, 10, 10])

In [61]:
burned_trace['n2']

array([3, 5, 5, ..., 5, 6, 5])

In [62]:
burned_trace['diff'].mean()

2.002029885057471

This is correct, we should see $0.1*20 = 2$ as mean

In [64]:
burned_trace['diff'].std()

2.9998288102924993

In [63]:
sum(np.where(burned_trace['diff']>0,1,0))/len(burned_trace['diff'])

0.6921229885057472

So actually it should be 69%, close to the normal value.....

## 4.4 Design an Experiment

This is the inverse of the previous question, I think this is asking for the 95% confidence interval

In [81]:
z = -1.65 #the z score for 95% one sided Normal distribution

In [83]:
sigma = np.sqrt(0.6*0.4+0.3*0.7)

N= np.power(sigma*z/(-0.1),2)

N

122.5125

We need roughly 123 trials

## 4.6 Hypothesis testing

In [88]:
with open('/home/jfyu/projects/ROS-Examples/Girls/girls.dat', 'r') as f:
    data = f.readlines()

In [89]:
data

['Proportion of girl births in 24 successive months in Vienna, 1908-1909, out of an average of 3900 births per month.  Data from Mises (1953). See Chapter 4 in Regression and Other Stories.\n',
 ' .4777\n',
 ' .4875\n',
 ' .4859\n',
 ' .4754\n',
 ' .4874\n',
 ' .4864\n',
 ' .4813\n',
 ' .4787\n',
 ' .4895\n',
 ' .4797\n',
 ' .4876\n',
 ' .4859\n',
 ' .4857\n',
 ' .4907\n',
 ' .5010\n',
 ' .4903\n',
 ' .4860\n',
 ' .4911\n',
 ' .4871\n',
 ' .4725\n',
 ' .4822\n',
 ' .4870\n',
 ' .4823\n',
 ' .4973']

In [90]:
data = data[1:]

In [91]:
data

[' .4777\n',
 ' .4875\n',
 ' .4859\n',
 ' .4754\n',
 ' .4874\n',
 ' .4864\n',
 ' .4813\n',
 ' .4787\n',
 ' .4895\n',
 ' .4797\n',
 ' .4876\n',
 ' .4859\n',
 ' .4857\n',
 ' .4907\n',
 ' .5010\n',
 ' .4903\n',
 ' .4860\n',
 ' .4911\n',
 ' .4871\n',
 ' .4725\n',
 ' .4822\n',
 ' .4870\n',
 ' .4823\n',
 ' .4973']

In [94]:
girl_birth = []
for i in data:
    girl_birth.append(np.float(i.strip('\n')))

In [118]:
df = pd.DataFrame({'girl_birth_rate':girl_birth})

In [119]:
df['total_monthly_girl_births'] = df['girl_birth_rate']*3900

In [120]:
df

,girl_birth_rate,total_monthly_girl_births
0,0.4777,1863.03
1,0.4875,1901.25
2,0.4859,1895.01
3,0.4754,1854.06
4,0.4874,1900.86
5,0.4864,1896.96
6,0.4813,1877.07
7,0.4787,1866.93
8,0.4895,1909.05
9,0.4797,1870.83


In [121]:
df['total_girl_births'] = df['total_monthly_girl_births'].cumsum()

In [122]:
df

,girl_birth_rate,total_monthly_girl_births,total_girl_births
0,0.4777,1863.03,1863.03
1,0.4875,1901.25,3764.28
2,0.4859,1895.01,5659.29
3,0.4754,1854.06,7513.35
4,0.4874,1900.86,9414.21
5,0.4864,1896.96,11311.17
6,0.4813,1877.07,13188.24
7,0.4787,1866.93,15055.17
8,0.4895,1909.05,16964.22
9,0.4797,1870.83,18835.05


In [142]:
df['extra_girl_births'] = df['total_monthly_girl_births'].diff().fillna(1863.03)

In [143]:
df

,girl_birth_rate,total_monthly_girl_births,total_girl_births,extra_girl_births,total_births,actual_variance,p_hat
0,0.4777,1863.03,1863.03,1863.03,3900,0.00799,0.477700
1,0.4875,1901.25,3764.28,38.22,7800,0.00799,0.482600
2,0.4859,1895.01,5659.29,-6.24,11700,0.00799,0.483700
3,0.4754,1854.06,7513.35,-40.95,15600,0.00799,0.481625
4,0.4874,1900.86,9414.21,46.80,19500,0.00799,0.482780
5,0.4864,1896.96,11311.17,-3.90,23400,0.00799,0.483383
6,0.4813,1877.07,13188.24,-19.89,27300,0.00799,0.483086
7,0.4787,1866.93,15055.17,-10.14,31200,0.00799,0.482538
8,0.4895,1909.05,16964.22,42.12,35100,0.00799,0.483311
9,0.4797,1870.83,18835.05,-38.22,39000,0.00799,0.482950


In [144]:
df['total_births'] = np.arange(1,len(df)+1)*3900

In [145]:
df

,girl_birth_rate,total_monthly_girl_births,total_girl_births,extra_girl_births,total_births,actual_variance,p_hat
0,0.4777,1863.03,1863.03,1863.03,3900,0.00799,0.477700
1,0.4875,1901.25,3764.28,38.22,7800,0.00799,0.482600
2,0.4859,1895.01,5659.29,-6.24,11700,0.00799,0.483700
3,0.4754,1854.06,7513.35,-40.95,15600,0.00799,0.481625
4,0.4874,1900.86,9414.21,46.80,19500,0.00799,0.482780
5,0.4864,1896.96,11311.17,-3.90,23400,0.00799,0.483383
6,0.4813,1877.07,13188.24,-19.89,27300,0.00799,0.483086
7,0.4787,1866.93,15055.17,-10.14,31200,0.00799,0.482538
8,0.4895,1909.05,16964.22,42.12,35100,0.00799,0.483311
9,0.4797,1870.83,18835.05,-38.22,39000,0.00799,0.482950


What's the standard deviation for these births?

In [146]:
df['actual_variance'] = np.std(df['extra_girl_births']/3900)

what would be expected if the birth proportions were constant? See p. 64

In [147]:
df['p_hat'] = df['total_girl_births']/df['total_births'].fillna(1863.03)

In [148]:
df

,girl_birth_rate,total_monthly_girl_births,total_girl_births,extra_girl_births,total_births,actual_variance,p_hat
0,0.4777,1863.03,1863.03,1863.03,3900,0.095621,0.477700
1,0.4875,1901.25,3764.28,38.22,7800,0.095621,0.482600
2,0.4859,1895.01,5659.29,-6.24,11700,0.095621,0.483700
3,0.4754,1854.06,7513.35,-40.95,15600,0.095621,0.481625
4,0.4874,1900.86,9414.21,46.80,19500,0.095621,0.482780
5,0.4864,1896.96,11311.17,-3.90,23400,0.095621,0.483383
6,0.4813,1877.07,13188.24,-19.89,27300,0.095621,0.483086
7,0.4787,1866.93,15055.17,-10.14,31200,0.095621,0.482538
8,0.4895,1909.05,16964.22,42.12,35100,0.095621,0.483311
9,0.4797,1870.83,18835.05,-38.22,39000,0.095621,0.482950


In [139]:
(df['p_hat']*(1-df['p_hat'])/df['extra_girl_births']).mean()

inf